In [1]:
import sys
import asyncio
from dotenv import load_dotenv

load_dotenv()

# Fix for Windows issues in Jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(), asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirect stderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr = sys.__stderr__

## Local MCP server

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import sys

client = MultiServerMCPClient(
    {
        "local_server": {
                "transport": "stdio",
                "command": sys.executable,
                "args": ["resources/2.1_mcp_server.py"],
            }
    }
)

In [3]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [4]:
from langchain.chat_models import init_chat_model
model = init_chat_model(model="meta-llama/llama-4-maverick-17b-128e-instruct",
                        model_provider="groq",
                        temperature = 0,
                        max_retries=3,
                        timeout=60,
                        max_tokens=300
                        )

In [5]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=prompt # type: ignore
)

In [6]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="Tell me about the langchain-mcp-adapters library")]},
    config=config # type: ignore
)

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='87c38655-d942-46f7-883f-6a5399f2f714'),
              AIMessage(content='{\n  "name": "search_web",\n  "parameters": {\n    "query": "langchain-mcp-adapters library"\n  }\n}', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 846, 'total_tokens': 879, 'completion_time': 0.053502038, 'completion_tokens_details': None, 'prompt_time': 0.019112528, 'prompt_tokens_details': None, 'queue_time': 0.049315122, 'total_time': 0.072614566}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3b96-dabb-7312-8ef7-13251f40f4df-0', usage_metadata={'input_tokens': 846, 'output_tokens': 33, 'total_tokens': 879})]}


## Online MCP

In [8]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [9]:
agent = create_agent(
    model=model,
    tools=tools,
)

In [10]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='6f6c207f-63dc-4821-9156-2c4b087e0099'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0dax89byy', 'function': {'arguments': '{"timezone":"America/New_York"}', 'name': 'get_current_time'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 907, 'total_tokens': 940, 'completion_time': 0.039072853, 'completion_tokens_details': None, 'prompt_time': 0.024116406, 'prompt_tokens_details': None, 'queue_time': 0.050076613, 'total_time': 0.063189259}, 'model_name': 'meta-llama/llama-4-maverick-17b-128e-instruct', 'system_fingerprint': 'fp_d2c1f7e199', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019b3b97-161b-7573-8725-b784a9af30af-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'America/New_York'}, 'id': '0dax89byy', 'type': 